In [23]:
import sys
import pandas as pd
import time
sys.path.append("ADD BASE DIR IF RUNNING LOCALLY")
pd.set_option('display.max_columns', 50)


In [13]:
import lionel.data_load.run
import lionel.model.run
import lionel.team.run
from lionel.data_load.storage_handler import StorageHandler

# Run the dataload

In [19]:
season = 25
next_gw = 1

sh = StorageHandler(local=True)
df_train = lionel.data_load.run.run(sh, season, next_gw)
df_train.head()

ERROR:lionel.data_load.ingestion.fpl.extract_fixtures:Invalid dtype for team_a_score. Expected int64, got float64.Attempting to convert...
ERROR:lionel.data_load.ingestion.fpl.extract_fixtures:Invalid dtype for team_h_score. Expected int64, got float64.Attempting to convert...
/Users/toby/Dev/lionel/lionel/data_load/process/process_train_data.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_gw = pd.concat([df_gw_previous, df_gw_current])
/Users/toby/Dev/lionel/lionel/data_load/process/process_train_data.py:225: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('

727


,is_home,season,gameweek,game_date,ds,game_complete,unique_id,assists,bps,creativity,...,opponent_team_name_Newcastle,opponent_team_name_Nottingham,opponent_team_name_Sheffield Utd,opponent_team_name_Southampton,opponent_team_name_Tottenham,opponent_team_name_West Ham,opponent_team_name_Wolves,position_FWD,position_GK,position_MID
0,True,24,1,2023-08-12,0,True,Aaron Ramsdale,0,10,0.0,...,False,True,False,False,False,False,False,False,True,False
1,True,24,1,2023-08-12,0,True,Benjamin White,0,8,7.3,...,False,True,False,False,False,False,False,False,False,False
2,True,24,1,2023-08-12,0,True,Bukayo Saka,0,37,46.2,...,False,True,False,False,False,False,False,False,False,True
3,True,24,1,2023-08-12,0,True,Cédric Alves Soares,0,0,0.0,...,False,True,False,False,False,False,False,False,False,False
4,True,24,1,2023-08-12,0,True,Declan Rice,0,18,4.6,...,False,True,False,False,False,False,False,False,False,True


# Run the models and make predictions

In [20]:
# set number of weeks ahead to predict
# this will account for double and missing gameweeks.
# E.g. if one team has a double gameweek it will include 
# six predictions in the next five gameweeks
gw_horizon = 5

# This will run all four models
s = time.time()
preds = lionel.model.run.run(sh, df_train, season, next_gw, gw_horizon)
print("Time taken: ", time.time() - s)

preds[preds.unique_id=="Bukayo Saka"]

INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (mps), used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name            | Type          | Params | Mode 
----------------------------------------------------------
0 | loss            | MAE           | 0      | train
1 | padder          | ConstantPad1d | 0      | train
2 | scaler          | TemporalNorm  | 0      | train
3 | hist_encoder    | LSTM          | 495 K  | train
4 | context_adapter | Linear        | 11.3 K | train
5 | mlp_decoder     | MLP           | 3.4 K  | train
6 | relu            | ReLU          | 0      | train
----------------------------------------------------------
509 K     Trainable params
0         Non-trainable params
509 K     Total params
2.040     Total estimated model p

Epoch 43:  48%|████▊     | 11/23 [00:00<00:00, 13.89it/s, v_num=75, train_loss_step=0.000, train_loss_epoch=0.500]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=1000` reached.


Epoch 43:  48%|████▊     | 11/23 [00:00<00:00, 13.84it/s, v_num=75, train_loss_step=0.000, train_loss_epoch=0.500]


/Users/toby/anaconda3/envs/lionel-forecasting/lib/python3.11/site-packages/neuralforecast/core.py:793: UserWarning: Dropped 24,090 unused rows from `futr_df`.
  warnings.warn(f"Dropped {dropped_rows:,} unused rows from `futr_df`.")
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (mps), used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


...

/Users/toby/anaconda3/envs/lionel-forecasting/lib/python3.11/site-packages/neuralforecast/core.py:199: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(
/Users/toby/anaconda3/envs/lionel-forecasting/lib/python3.11/site-packages/mlforecast/core.py:417: UserWarning: The following series were dropped completely due to the transformations and features: ['Billy Crellin', 'Kadan Young', 'Lander Emery', 'Yunus Konak'].
These series won't show up if you use `MLForecast.forecast_fitted_values()`.
You can set `dropna=False` or use transformations that require less samples to mitigate this
  warnings.warn(
/Users/toby/anaconda3/envs/lionel-forecasting/lib/python3.11/site-packages/coreforecast/lag_transforms.py:346: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(self.stats_[:, 2] / (n - 1))
/Users/toby/anaconda3/envs/lio

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000308 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 836
[LightGBM] [Info] Number of data points in the train set: 23296, number of used features: 9
[LightGBM] [Info] Start training from score 1.124313


/Users/toby/anaconda3/envs/lionel-forecasting/lib/python3.11/site-packages/coreforecast/lag_transforms.py:346: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(self.stats_[:, 2] / (n - 1))
/Users/toby/anaconda3/envs/lionel-forecasting/lib/python3.11/site-packages/mlforecast/core.py:594: UserWarning: Found null values in expanding_std_lag1, exponentially_weighted_mean_lag1_alpha0.1, exponentially_weighted_mean_lag1_alpha0.5.
  warnings.warn(f'Found null values in {", ".join(cols_with_nulls)}.')
/Users/toby/anaconda3/envs/lionel-forecasting/lib/python3.11/site-packages/mlforecast/core.py:594: UserWarning: Found null values in exponentially_weighted_mean_lag1_alpha0.1, exponentially_weighted_mean_lag1_alpha0.5.
  warnings.warn(f'Found null values in {", ".join(cols_with_nulls)}.')
/Users/toby/anaconda3/envs/lionel-forecasting/lib/python3.11/site-packages/mlforecast/core.py:594: UserWarning: Found null values in exponentially_weighted_mean_lag1_alpha0.1, exponentially_wei

Time taken:  84.59511184692383


,unique_id,gameweek,Naive,LGBMRegressor_no_exog,LGBMRegressor_with_exog,LSTMWithReLU,season
490,Bukayo Saka,1,0.0,2.877435,3.814077,5.0,25
491,Bukayo Saka,2,0.0,4.671797,4.964903,5.0,25
492,Bukayo Saka,3,0.0,3.541000,6.469857,5.0,25
493,Bukayo Saka,4,0.0,3.151870,4.378277,5.0,25
494,Bukayo Saka,5,0.0,4.631508,6.003738,5.0,25


# Run the team selection using the predictions

In [24]:
# Selects four teams - one with each set of preds
df_team = lionel.team.run.run(sh, season, next_gw)

pred_var = "LGBMRegressor_with_exog"
df_team[df_team[f"picked_{pred_var}"] == 1]

/Users/toby/Dev/lionel/lionel/utils.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[prefix] = pd.from_dummies(df[cols], default_category=default)
/Users/toby/Dev/lionel/lionel/utils.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[prefix] = df[prefix].str.split("_").str[-1]
/Users/toby/anaconda3/envs/lionel-forecasting/lib/python3.11/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Conve

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/toby/anaconda3/envs/lionel-forecasting/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/6h/pq_v8g215v52_jm4968ld4080000gn/T/c8b99ce94e174b2da83d19099ce37fe8-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/6h/pq_v8g215v52_jm4968ld4080000gn/T/c8b99ce94e174b2da83d19099ce37fe8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1466 COLUMNS
At line 41152 RHS
At line 42614 BOUNDS
At line 44069 ENDATA
Problem MODEL has 1461 rows, 1454 columns and 36297 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 223 - 0.00 seconds
Cgl0004I processed model has 751 rows, 1454 columns (1454 integer (1454 of which binary)) and 5089 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I 

/Users/toby/anaconda3/envs/lionel-forecasting/lib/python3.11/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
/Users/toby/Dev/lionel/lionel/team/select.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team["first_xi"] = 0
/Users/toby/anaconda3/envs/lionel-forecasting/lib/python3.11/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/toby/anaconda3/envs/lionel-forecasting/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/6h/pq_v8g215v52_jm4968ld4080000gn/T/e50965875fa944d9adb505dc43df3e4e-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/6h/pq_v8g215v52_jm4968ld4080000gn/T/e50965875fa944d9adb505dc43df3e4e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1466 COLUMNS
At line 42126 RHS
At line 43588 BOUNDS
At line 45043 ENDATA
Problem MODEL has 1461 rows, 1454 columns and 36297 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 82.4404 - 0.00 seconds
Cgl0004I processed model has 751 rows, 1454 columns (1454 integer (1454 of which binary)) and 5089 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.857143
Cbc0038I Solution found of -82.3952
Cbc0038I B

/Users/toby/anaconda3/envs/lionel-forecasting/lib/python3.11/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
/Users/toby/Dev/lionel/lionel/team/select.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team["first_xi"] = 0
/Users/toby/anaconda3/envs/lionel-forecasting/lib/python3.11/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/toby/anaconda3/envs/lionel-forecasting/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/6h/pq_v8g215v52_jm4968ld4080000gn/T/474fef7babd843f684d9439b4285e265-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/6h/pq_v8g215v52_jm4968ld4080000gn/T/474fef7babd843f684d9439b4285e265-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1466 COLUMNS
At line 42126 RHS
At line 43588 BOUNDS
At line 45043 ENDATA
Problem MODEL has 1461 rows, 1454 columns and 36297 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 81.1809 - 0.00 seconds
Cgl0004I processed model has 751 rows, 1454 columns (1454 integer (1454 of which binary)) and 5089 elements
Cbc0038I Initial state - 4 integers unsatisfied sum - 0.9375
Cbc0038I Pass   1: suminf.    0.51724 (2) obj. -

/Users/toby/anaconda3/envs/lionel-forecasting/lib/python3.11/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
/Users/toby/Dev/lionel/lionel/team/select.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team["first_xi"] = 0
/Users/toby/anaconda3/envs/lionel-forecasting/lib/python3.11/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/toby/anaconda3/envs/lionel-forecasting/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/6h/pq_v8g215v52_jm4968ld4080000gn/T/b84c3e2c848d45d9a3287650e800e378-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/6h/pq_v8g215v52_jm4968ld4080000gn/T/b84c3e2c848d45d9a3287650e800e378-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1466 COLUMNS
At line 41170 RHS
At line 42632 BOUNDS
At line 44087 ENDATA
Problem MODEL has 1461 rows, 1454 columns and 36297 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 58.625 - 0.00 seconds
Cgl0004I processed model has 751 rows, 1454 columns (1454 integer (1454 of which binary)) and 5089 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.375
Cb

/Users/toby/anaconda3/envs/lionel-forecasting/lib/python3.11/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
/Users/toby/Dev/lionel/lionel/team/select.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team["first_xi"] = 0


,unique_id,team_name,position,value,pred_Naive,pred_LGBMRegressor_no_exog,pred_LGBMRegressor_with_exog,pred_LSTMWithReLU,pred_season,picked_Naive,captain_Naive,first_xi_Naive,season,picked_time,picked_LGBMRegressor_no_exog,captain_LGBMRegressor_no_exog,first_xi_LGBMRegressor_no_exog,picked_LGBMRegressor_with_exog,captain_LGBMRegressor_with_exog,first_xi_LGBMRegressor_with_exog,picked_LSTMWithReLU,captain_LSTMWithReLU,first_xi_LSTMWithReLU
2,Bruno Guimarães Rodriguez Moura,Newcastle,MID,58,16.0,5.171441,5.916905,2.0,25.0,1.0,0,1.0,25,2024-08-17 15:41:35.018291,1.0,0,1.0,1.0,0,1.0,0.0,0,NaN
7,Jarell Quansah,Liverpool,DEF,40,15.0,3.777053,3.685713,0.0,25.0,1.0,0,1.0,25,2024-08-17 15:41:35.018291,1.0,0,0.0,1.0,0,0.0,0.0,0,NaN
70,Ashley Young,Everton,DEF,44,1.0,2.721493,4.120046,1.0,25.0,0.0,0,NaN,25,2024-08-17 15:41:35.018291,0.0,0,NaN,1.0,0,0.0,0.0,0,NaN
138,Cole Palmer,Chelsea,MID,63,6.0,6.117993,6.423853,6.0,25.0,0.0,0,NaN,25,2024-08-17 15:41:35.018291,1.0,0,1.0,1.0,0,1.0,1.0,1,1.0
206,Erling Haaland,Manchester City,FWD,143,2.0,6.644238,6.767150,4.0,25.0,0.0,0,NaN,25,2024-08-17 15:41:35.018291,1.0,1,1.0,1.0,1,1.0,0.0,0,NaN
237,Gabriel dos Santos Magalhães,Arsenal,DEF,54,1.0,3.787965,4.266443,2.0,25.0,0.0,0,NaN,25,2024-08-17 15:41:35.018291,0.0,0,NaN,1.0,0,1.0,0.0,0,NaN
265,Heung-Min Son,Tottenham,MID,100,7.0,5.128929,5.095902,3.0,25.0,0.0,0,NaN,25,2024-08-17 15:41:35.018291,1.0,0,1.0,1.0,0,1.0,0.0,0,NaN
356,Jordan Pickford,Everton,GK,48,1.0,3.203331,3.448782,2.0,25.0,0.0,0,NaN,25,2024-08-17 15:41:35.018291,0.0,0,NaN,1.0,0,1.0,0.0,0,NaN
373,Joško Gvardiol,Manchester City,DEF,52,2.0,4.553040,4.552625,2.0,25.0,0.0,0,NaN,25,2024-08-17 15:41:35.018291,1.0,0,1.0,1.0,0,1.0,0.0,0,NaN
475,Martin Ødegaard,Arsenal,MID,86,10.0,4.121999,5.554430,3.0,25.0,0.0,0,NaN,25,2024-08-17 15:41:35.018291,0.0,0,NaN,1.0,0,1.0,0.0,0,NaN
